In [1]:
import pandas as pd
df = pd.read_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_train_dsr1_20240317.xlsx')
df

,input,r1_thinking,r1_answer
0,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户提供的广告ID是14160088313和36557971，他们询问“哪里有医疗场景”...,您好！关于广告ID 14160088313（子ID 36557971）中涉及医疗场景的审核问...
1,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“目前腾讯广告平台支持哪些设备的定向”。首先，我需要查看提供...,腾讯广告平台支持以下设备定向功能：\n\n### 一、设备品牌及型号定向\n支持**90+主...
2,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户之前问过关于扩量种子人群未开启的问题，现在他们再次反馈在后台找不到这个选项。我需要...,您好！关于您提到的找不到「扩量种子人群」选项的问题，以下是可能的原因及操作指引：\n\n--...
3,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“复制建的广告会继承原来的广告的数据模型嘛”。首先，我要查看...,是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到...
4,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户问的是“扩量种子人群：未开启 在哪里选择？”。首先，我需要理解用户的问题。他们可能在...,在腾讯广告中配置扩量种子人群，需按以下步骤操作：\n\n---\n\n### 📌 扩量种子人...
...,...,...,...
483,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好，用户的问题是：“这个搜索扩量是版位扩量吗还是？”我需要先理解用户的问题核心。他们想知道搜...,**关于广告ID 12682513045的搜索场景扩量版位问题解答**\n\n您提到的**搜...
484,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是“账户id是什么？”，我需要根据提供的广告资料来回答。首先查看广告资料中的相...,**账户ID是腾讯广告账户的唯一标识符**，用于精准识别和管理广告账户。\n\n---\n\...
485,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是如何查看电脑端或移动端的消费。我需要参考广告资料里的信息来回答这个问题。首先...,以下是查看电脑端和移动端广告消费的详细方式：\n\n---\n\n### 一、电脑端查看消费...
486,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问“怎么看组织的主管理员”，我需要根据提供的广告资料来回答。首先，我应该查看广告资...,根据腾讯广告的规则，组织的主管理员通常是企业法人或营销负责人。查看主管理员的方式如下：\n\...


In [2]:
df.columns

Index(['input', 'r1_thinking', 'r1_answer'], dtype='object')

In [14]:
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse

# 设置请求超时时间（秒）
REQUEST_TIMEOUT = 5

def check_link_accessibility(url):
    """检查单个链接的可访问性"""
    try:
        if "example.com" in url:
            return False, "示例域名"
        # 添加User-Agent头以避免被某些网站拒绝
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # 只获取头部信息，不下载整个页面
        response = requests.head(
            url, 
            headers=headers, 
            timeout=REQUEST_TIMEOUT,
            allow_redirects=True
        )
        
        # 检查状态码是否为2xx或3xx
        if response.status_code < 400:
            return True, None
        else:
            return False, f"HTTP {response.status_code}"
    
    except requests.exceptions.RequestException as e:
        error_msg = str(e)
        if 'SSL' in error_msg:
            return False, "SSL错误"
        elif 'ConnectTimeout' in error_msg:
            return False, "连接超时"
        elif 'ConnectionError' in error_msg:
            return False, "连接错误"
        else:
            return False, error_msg
        
def check_links_accessibility(urls):
    """批量检查链接可访问性（使用多线程）"""
    if not urls:
        return [], []
    
    accessible = []
    inaccessible = []
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(check_link_accessibility, urls))
    
    for url, (is_accessible, error_msg) in zip(urls, results):
        if is_accessible:
            accessible.append(url)
        else:
            inaccessible.append((url, error_msg))
    
    return accessible, inaccessible

def extract_links(text):
    """从文本中提取所有超链接"""
    if pd.isna(text):
        return []
    
    # markdown_links = re.findall(r'(https?://[^\s)]+)', str(text))
    markdown_links = re.findall( r'[($$](https?://[^\s)$$]+)[)\]]', str(text))

    
    return markdown_links

# 从r1_answer列提取所有超链接
df['all_links'] = df['r1_answer'].apply(extract_links)

# 检查每个链接是否出现在input列中
def check_links(row):
    if not row['extracted_links']:
        return True
    input_text = str(row['input']) if pd.notna(row['input']) else ''
    return all(link in input_text for link in row['extracted_links'])

def check_and_record_links(row):
    if not row['all_links']:
        return {'link_check': True, 'new_links': []}
    
    input_text = str(row['input']) if pd.notna(row['input']) else ''
    missing_links = [link for link in row['all_links'] if link not in input_text]
    
    # 检查缺失链接的可访问性
    accessible, inaccessible = check_links_accessibility(missing_links)
    
    return {
        'link_check': len(missing_links) == 0,
        'new_links': missing_links,
        'accessible_links': accessible,
        'inaccessible_links': inaccessible
    }

result = df.apply(check_and_record_links, axis=1, result_type='expand')
df['link_check'] = result['link_check']
df['new_links'] = result['new_links']
df['accessible_links'] = result['accessible_links']
df['inaccessible_links'] = result['inaccessible_links']
# df['link_check'] = df.apply(check_links, axis=1)

# 删除临时列（如果需要）
# df = df.drop('extracted_links', axis=1)

df

/Users/wenweili/PycharmProjects/OpenZero/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


,input,r1_thinking,r1_answer,link_check,extracted_links,all_links,new_links,accessible_links,inaccessible_links
0,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户提供的广告ID是14160088313和36557971，他们询问“哪里有医疗场景”...,您好！关于广告ID 14160088313（子ID 36557971）中涉及医疗场景的审核问...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[]
1,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“目前腾讯广告平台支持哪些设备的定向”。首先，我需要查看提供...,腾讯广告平台支持以下设备定向功能：\n\n### 一、设备品牌及型号定向\n支持**90+主...,True,[https://e.qq.com/ads/helpcenter/detail?cid=50...,[https://e.qq.com/ads/helpcenter/detail?cid=50...,[],[],[]
2,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户之前问过关于扩量种子人群未开启的问题，现在他们再次反馈在后台找不到这个选项。我需要...,您好！关于您提到的找不到「扩量种子人群」选项的问题，以下是可能的原因及操作指引：\n\n--...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[],[],[]
3,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“复制建的广告会继承原来的广告的数据模型嘛”。首先，我要查看...,是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://docs.qq.com/ad],[],"[(https://docs.qq.com/ad, HTTP 404)]"
4,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户问的是“扩量种子人群：未开启 在哪里选择？”。首先，我需要理解用户的问题。他们可能在...,在腾讯广告中配置扩量种子人群，需按以下步骤操作：\n\n---\n\n### 📌 扩量种子人...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://zhishu.tencent.com/],[],"[(https://zhishu.tencent.com/, HTTPSConnection..."
...,...,...,...,...,...,...,...,...,...
483,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好，用户的问题是：“这个搜索扩量是版位扩量吗还是？”我需要先理解用户的问题核心。他们想知道搜...,**关于广告ID 12682513045的搜索场景扩量版位问题解答**\n\n您提到的**搜...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2024/4...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/4...,[],[],[]
484,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是“账户id是什么？”，我需要根据提供的广告资料来回答。首先查看广告资料中的相...,**账户ID是腾讯广告账户的唯一标识符**，用于精准识别和管理广告账户。\n\n---\n\...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[]
485,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是如何查看电脑端或移动端的消费。我需要参考广告资料里的信息来回答这个问题。首先...,以下是查看电脑端和移动端广告消费的详细方式：\n\n---\n\n### 一、电脑端查看消费...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/3...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/3...,[],[],[]
486,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问“怎么看组织的主管理员”，我需要根据提供的广告资料来回答。首先，我应该查看广告资...,根据腾讯广告的规则，组织的主管理员通常是企业法人或营销负责人。查看主管理员的方式如下：\n\...,True,"[https://ad.qq.com/cm/home, https://doc.weixin...","[https://ad.qq.com/cm/home, https://doc.weixin...",[],[],[]


In [15]:
df.to_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_train_dsr1_20240317_link_check.xlsx', index=False)

In [10]:
df['link_check'].value_counts()

link_check
True     319
False    169
Name: count, dtype: int64

# ds改写，去除幻觉链接

In [35]:
# 调用火山引擎 api
import requests
import time
import random
import numpy as np
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor

ARK_API_KEY = "e249abdf-51d4-482a-b5c8-8777a73990c2"
client = OpenAI(
    api_key = ARK_API_KEY,
    base_url = "https://ark.cn-beijing.volces.com/api/v3",
)


def request_deepseek(prompt, need_thinking=False):
    # prompt = prompt_t.replace("{query}", prompt)
    response = client.chat.completions.create(
        # model="ep-20250212141905-2grjj",  # deeepseek v3
        # model="ep-20250227113433-vv7hr",  # deeepseek r1
        model="ep-20250326193223-b2b4d",  # deepseek-v3-0324
        messages=[
            # {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt},
        ],
        stream=False,
        temperature=0.0
    )
    if need_thinking:
        result = ("**Thinking**:\n" + response.choices[0].message.reasoning_content, '**Answer**:\n' + \
                         response.choices[0].message.content)
    else:
        result =  response.choices[0].message.content

    return result

def concurrent_requests(prompts, num_requests):
    answers = []
    with ThreadPoolExecutor(max_workers=num_requests) as executor:
        futures = {executor.submit(request_deepseek, prompts[i]): i for i in range(num_requests)}
        for future in futures:
            result = future.result()
            if result is not None:
                answers.append(result)
    return answers

# prompts = ['南极为啥没有北极熊，一句话回答', '介绍林黛玉', '天空网是啥', '写篇父亲节的作文，800字的', 
#            '如何看待中美战争', '德国为啥战败', '写个df删除2列的代码', 'java写个冒泡排序']

# num_requests = len(prompts)  # 设置并发请求数量
# answers = concurrent_requests(prompts, num_requests)
# print(answers)
request_deepseek("你好")


'你好！😊 很高兴见到你～有什么我可以帮你的吗？'

In [39]:
prompt_t = '''
我用大模型生成了[模型回答]，但其中存在一些模型自己编造的链接[幻觉链接]，请帮我将这些[幻觉链接]及其相关的文本在[模型回答]中去掉，并改写通顺，返回答案时直接输出改写后的文本即可，不需要加任何解释说明。
[模型回答]=
{answer}

[幻觉链接]=
{links}
'''

def gen_prompt(answer, links):
    return prompt_t.replace("{answer}", str(answer)).replace("{links}", str(links))

test_a = '''
是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到新广告上，帮助新广告快速起量。这是通过批量复制功能实现的，具体原理如下：

---

### 📌 数据模型继承机制
通过批量复制功能生成的新广告，会直接复用原广告的历史投放数据模型（如转化率、CTR等核心指标），有效缩短新广告的冷启动周期。

---

### 📋 批量复制操作指引
1. **功能入口**  
   广告管理列表 > 点击原广告操作列【更多】>【批量复制】  
   ![批量复制入口](https://bbx-pic.gtimg.com/bbx/pictures/2023/211_20231012165051_243654.png)

2. **定向与创意设置**  
   支持通过叉乘组合生成多维度广告：
   - 定向变更项（最多4组）  
   - 创意变更项（可批量设置新素材/文案）  
   ![定向设置示例](https://bbx-pic.gtimg.com/bbx/pictures/2020/81_20200529210134_753232.png)

---

### ⚡ 最佳实践建议
- 优先复制**过去7天内有转化数据**的广告
- 批量复制后建议采用"原出价±10%"的阶梯出价策略
- 新广告需至少保持与原广告50%的创意相似度

> 📘 相关文档：[批量广告操作指南](https://docs.qq.com/ad)  
（文档更新时间：2023.3.7）
'''

test_l = ['https://docs.qq.com/ad']
test_p = gen_prompt(test_a, test_l)
request_deepseek(test_p)

'是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到新广告上，帮助新广告快速起量。这是通过批量复制功能实现的，具体原理如下：\n\n---\n\n### 📌 数据模型继承机制\n通过批量复制功能生成的新广告，会直接复用原广告的历史投放数据模型（如转化率、CTR等核心指标），有效缩短新广告的冷启动周期。\n\n---\n\n### 📋 批量复制操作指引\n1. **功能入口**  \n   广告管理列表 > 点击原广告操作列【更多】>【批量复制】  \n\n2. **定向与创意设置**  \n   支持通过叉乘组合生成多维度广告：\n   - 定向变更项（最多4组）  \n   - 创意变更项（可批量设置新素材/文案）  \n\n---\n\n### ⚡ 最佳实践建议\n- 优先复制**过去7天内有转化数据**的广告\n- 批量复制后建议采用"原出价±10%"的阶梯出价策略\n- 新广告需至少保持与原广告50%的创意相似度'

In [40]:
from tqdm import *

i = 0
for index, row in tqdm(df.iterrows()):
    if row['link_check'] == False:
        answer = row['r1_answer']
        links = row['new_links']
        prompt = gen_prompt(answer, links)
        result = request_deepseek(prompt)
        if i < 3:
            print(result)
        i += 1
        df.at[index, 'answer_new'] = result
    else:
        df.at[index, 'answer_new'] = row['r1_answer']
df

4it [00:08,  2.03s/it]

是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到新广告上，帮助新广告快速起量。这是通过批量复制功能实现的，具体原理如下：

---

### 📌 数据模型继承机制
通过批量复制功能生成的新广告，会直接复用原广告的历史投放数据模型（如转化率、CTR等核心指标），有效缩短新广告的冷启动周期。

---

### 📋 批量复制操作指引
1. **功能入口**  
   广告管理列表 > 点击原广告操作列【更多】>【批量复制】  

2. **定向与创意设置**  
   支持通过叉乘组合生成多维度广告：
   - 定向变更项（最多4组）  
   - 创意变更项（可批量设置新素材/文案）  

---

### ⚡ 最佳实践建议
- 优先复制**过去7天内有转化数据**的广告
- 批量复制后建议采用"原出价±10%"的阶梯出价策略
- 新广告需至少保持与原广告50%的创意相似度


5it [00:18,  4.17s/it]

在腾讯广告中配置扩量种子人群，需按以下步骤操作：

---

### 📌 扩量种子人群配置路径
1. **进入广告层级设置**  
   在广告创建或编辑页，找到「定向设置」模块，点击「智能定向」右侧的「设置」按钮。

2. **开启自动扩量功能**  
   在弹窗中勾选「开启自动扩量」，此时系统会默认使用广告历史转化数据作为种子人群。

3. **自定义种子人群（可选）**  
   若需手动指定种子人群：
   - 需提前创建人群包
   - 在「自动扩量」设置弹窗中选择「自定义种子人群」并关联已建人群包

---

### ❗ 重点说明
- **未开启状态**  
  若显示「扩量种子人群：未开启」，表示未手动指定种子人群包，系统将自动使用广告账户的 **历史转化数据** 作为扩量依据。
  
- **行为兴趣定向冲突**  
  自动扩量功能 **不可与“行为兴趣意向”定向同时使用**（2023年6月23日后生效）。

---

### 📊 建议操作流程
```mermaid
graph TD
    A[创建人群包] --> B(广告层级开启自动扩量)
    B --> C{是否自定义种子人群?}
    C -- 是 --> D[关联人群包]
    C -- 否 --> E[系统自动使用历史数据]
```


7it [00:26,  4.17s/it]

根据您的投放场景（微信搜索竞价广告）和广告形态描述，您提到的“框起来的广告”可能不是您当前投放的搜索广告。以下是关键信息说明：

### 广告形态差异说明

1. **投放位置差异**
   - 您投放的微信搜索广告为搜索竞价广告（CPC/OCPC）
   - 截图中展示的是朋友圈出框式广告（属于品牌广告）

2. **创意形态差异**
   | 广告类型 | 展示位置 | 创意特点 |
   |---|---|---|
   | 微信搜索广告 | 搜索结果页 | 文字链/简约图文 |
   | 朋友圈出框式广告 | 朋友圈信息流 | 突破边框的立体创意 |

3. **快速验证方法**
   ▶️ 登录腾讯广告平台检查：
   - 账户ID 25587802的投放位置是否包含"朋友圈"
   - 创意组件是否添加了3D素材/异形素材
   - 广告版位是否勾选"朋友圈"（出框式广告为朋友圈专属形态）

建议通过广告平台-创意预览功能，输入广告ID查看实际投放样式。若需进一步确认，可提供广告截图（隐去敏感信息）或广告ID帮您核查。


488it [30:05,  3.70s/it]


,input,r1_thinking,r1_answer,link_check,extracted_links,all_links,new_links,accessible_links,inaccessible_links,answer_new
0,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户提供的广告ID是14160088313和36557971，他们询问“哪里有医疗场景”...,您好！关于广告ID 14160088313（子ID 36557971）中涉及医疗场景的审核问...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[],您好！关于广告ID 14160088313（子ID 36557971）中涉及医疗场景的审核问...
1,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“目前腾讯广告平台支持哪些设备的定向”。首先，我需要查看提供...,腾讯广告平台支持以下设备定向功能：\n\n### 一、设备品牌及型号定向\n支持**90+主...,True,[https://e.qq.com/ads/helpcenter/detail?cid=50...,[https://e.qq.com/ads/helpcenter/detail?cid=50...,[],[],[],腾讯广告平台支持以下设备定向功能：\n\n### 一、设备品牌及型号定向\n支持**90+主...
2,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户之前问过关于扩量种子人群未开启的问题，现在他们再次反馈在后台找不到这个选项。我需要...,您好！关于您提到的找不到「扩量种子人群」选项的问题，以下是可能的原因及操作指引：\n\n--...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[],[],[],您好！关于您提到的找不到「扩量种子人群」选项的问题，以下是可能的原因及操作指引：\n\n--...
3,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“复制建的广告会继承原来的广告的数据模型嘛”。首先，我要查看...,是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://docs.qq.com/ad],[],"[(https://docs.qq.com/ad, HTTP 404)]",是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到...
4,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户问的是“扩量种子人群：未开启 在哪里选择？”。首先，我需要理解用户的问题。他们可能在...,在腾讯广告中配置扩量种子人群，需按以下步骤操作：\n\n---\n\n### 📌 扩量种子人...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://zhishu.tencent.com/],[],"[(https://zhishu.tencent.com/, HTTPSConnection...",在腾讯广告中配置扩量种子人群，需按以下步骤操作：\n\n---\n\n### 📌 扩量种子人...
...,...,...,...,...,...,...,...,...,...,...
483,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好，用户的问题是：“这个搜索扩量是版位扩量吗还是？”我需要先理解用户的问题核心。他们想知道搜...,**关于广告ID 12682513045的搜索场景扩量版位问题解答**\n\n您提到的**搜...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2024/4...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/4...,[],[],[],**关于广告ID 12682513045的搜索场景扩量版位问题解答**\n\n您提到的**搜...
484,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是“账户id是什么？”，我需要根据提供的广告资料来回答。首先查看广告资料中的相...,**账户ID是腾讯广告账户的唯一标识符**，用于精准识别和管理广告账户。\n\n---\n\...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[],**账户ID是腾讯广告账户的唯一标识符**，用于精准识别和管理广告账户。\n\n---\n\...
485,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是如何查看电脑端或移动端的消费。我需要参考广告资料里的信息来回答这个问题。首先...,以下是查看电脑端和移动端广告消费的详细方式：\n\n---\n\n### 一、电脑端查看消费...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/3...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/3...,[],[],[],以下是查看电脑端和移动端广告消费的详细方式：\n\n---\n\n### 一、电脑端查看消费...
486,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问“怎么看组织的主管理员”，我需要根据提供的广告资料来回答。首先，我应该查看广告资...,根据腾讯广告的规则，组织的主管理员通常是企业法人或营销负责人。查看主管理员的方式如下：\n\...,True,"[https://ad.qq.com/cm/home, https://doc.weixin...","[https://ad.qq.com/cm/home, https://doc.weixin...",[],[],[],根据腾讯广告的规则，组织的主管理员通常是企业法人或营销负责人。查看主管理员的方式如下：\n\...


In [41]:
df.to_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_train_dsr1_20240317_remove_hallucination_link.xlsx', index=False)

In [42]:
ndf = df.copy()

In [50]:
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse

# 设置请求超时时间（秒）
REQUEST_TIMEOUT = 5

def check_link_accessibility(url):
    """检查单个链接的可访问性"""
    try:
        # 添加User-Agent头以避免被某些网站拒绝
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # 只获取头部信息，不下载整个页面
        response = requests.head(
            url, 
            headers=headers, 
            timeout=REQUEST_TIMEOUT,
            allow_redirects=True
        )
        
        # 检查状态码是否为2xx或3xx
        if response.status_code < 400:
            return True, None
        else:
            return False, f"HTTP {response.status_code}"
    
    except requests.exceptions.RequestException as e:
        error_msg = str(e)
        if 'SSL' in error_msg:
            return False, "SSL错误"
        elif 'ConnectTimeout' in error_msg:
            return False, "连接超时"
        elif 'ConnectionError' in error_msg:
            return False, "连接错误"
        else:
            return False, error_msg
        
def check_links_accessibility(urls):
    """批量检查链接可访问性（使用多线程）"""
    if not urls:
        return [], []
    
    accessible = []
    inaccessible = []
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(check_link_accessibility, urls))
    
    for url, (is_accessible, error_msg) in zip(urls, results):
        if is_accessible:
            accessible.append(url)
        else:
            inaccessible.append((url, error_msg))
    
    return accessible, inaccessible

def extract_links(text):
    """从文本中提取所有超链接"""
    if pd.isna(text):
        return []
    
    # markdown_links = re.findall(r'(https?://[^\s)]+)', str(text))
    markdown_links = re.findall( r'[($$](https?://[^\s)$$]+)[)\]]', str(text))
    # markdown_links = re.findall(r'$\s*(https?://[^)\s]+)\s*$', str(text))

    
    
    return markdown_links

# 从r1_answer列提取所有超链接
ndf['all_links'] = ndf['answer_new'].apply(extract_links)

# 检查每个链接是否出现在input列中
def check_links(row):
    if not row['extracted_links']:
        return True
    input_text = str(row['input']) if pd.notna(row['input']) else ''
    return all(link in input_text for link in row['extracted_links'])

def check_and_record_links(row):
    if not row['all_links']:
        return {'link_check': True, 'new_links': []}
    
    input_text = str(row['input']) if pd.notna(row['input']) else ''
    missing_links = [link for link in row['all_links'] if link not in input_text]
    
    # 检查缺失链接的可访问性
    # accessible, inaccessible = check_links_accessibility(missing_links)
    
    return {
        'link_check': len(missing_links) == 0,
        'new_links': missing_links,
    }

result = ndf.apply(check_and_record_links, axis=1, result_type='expand')
ndf['link_check'] = result['link_check']
ndf['new_links'] = result['new_links']
# df['link_check'] = df.apply(check_links, axis=1)

# 删除临时列（如果需要）
# df = df.drop('extracted_links', axis=1)

ndf

,input,r1_thinking,r1_answer,link_check,extracted_links,all_links,new_links,accessible_links,inaccessible_links,answer_new
0,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户提供的广告ID是14160088313和36557971，他们询问“哪里有医疗场景”...,您好！关于广告ID 14160088313（子ID 36557971）中涉及医疗场景的审核问...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[],您好！关于广告ID 14160088313（子ID 36557971）中涉及医疗场景的审核问...
1,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“目前腾讯广告平台支持哪些设备的定向”。首先，我需要查看提供...,腾讯广告平台支持以下设备定向功能：\n\n### 一、设备品牌及型号定向\n支持**90+主...,True,[https://e.qq.com/ads/helpcenter/detail?cid=50...,[https://e.qq.com/ads/helpcenter/detail?cid=50...,[],[],[],腾讯广告平台支持以下设备定向功能：\n\n### 一、设备品牌及型号定向\n支持**90+主...
2,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户之前问过关于扩量种子人群未开启的问题，现在他们再次反馈在后台找不到这个选项。我需要...,您好！关于您提到的找不到「扩量种子人群」选项的问题，以下是可能的原因及操作指引：\n\n--...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[],[],[],您好！关于您提到的找不到「扩量种子人群」选项的问题，以下是可能的原因及操作指引：\n\n--...
3,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，我现在需要处理用户的问题：“复制建的广告会继承原来的广告的数据模型嘛”。首先，我要查看...,是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[],"[(https://docs.qq.com/ad, HTTP 404)]",是的，复制创建的广告会继承原广告的数据模型。腾讯广告系统会自动将历史广告的优质数据模型复用到...
4,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,嗯，用户问的是“扩量种子人群：未开启 在哪里选择？”。首先，我需要理解用户的问题。他们可能在...,在腾讯广告中配置扩量种子人群，需按以下步骤操作：\n\n---\n\n### 📌 扩量种子人...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,[],[],[],"[(https://zhishu.tencent.com/, HTTPSConnection...",在腾讯广告中配置扩量种子人群，需按以下步骤操作：\n\n---\n\n### 📌 扩量种子人...
...,...,...,...,...,...,...,...,...,...,...
483,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好，用户的问题是：“这个搜索扩量是版位扩量吗还是？”我需要先理解用户的问题核心。他们想知道搜...,**关于广告ID 12682513045的搜索场景扩量版位问题解答**\n\n您提到的**搜...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2024/4...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/4...,[],[],[],**关于广告ID 12682513045的搜索场景扩量版位问题解答**\n\n您提到的**搜...
484,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是“账户id是什么？”，我需要根据提供的广告资料来回答。首先查看广告资料中的相...,**账户ID是腾讯广告账户的唯一标识符**，用于精准识别和管理广告账户。\n\n---\n\...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/2...,[],[],[],**账户ID是腾讯广告账户的唯一标识符**，用于精准识别和管理广告账户。\n\n---\n\...
485,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问的是如何查看电脑端或移动端的消费。我需要参考广告资料里的信息来回答这个问题。首先...,以下是查看电脑端和移动端广告消费的详细方式：\n\n---\n\n### 一、电脑端查看消费...,True,[https://bbx-pic.gtimg.com/bbx/pictures/2023/3...,[https://bbx-pic.gtimg.com/bbx/pictures/2023/3...,[],[],[],以下是查看电脑端和移动端广告消费的详细方式：\n\n---\n\n### 一、电脑端查看消费...
486,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,好的，用户问“怎么看组织的主管理员”，我需要根据提供的广告资料来回答。首先，我应该查看广告资...,根据腾讯广告的规则，组织的主管理员通常是企业法人或营销负责人。查看主管理员的方式如下：\n\...,True,"[https://ad.qq.com/cm/home, https://doc.weixin...","[https://ad.qq.com/cm/home, https://doc.weixin...",[],[],[],根据腾讯广告的规则，组织的主管理员通常是企业法人或营销负责人。查看主管理员的方式如下：\n\...


In [46]:
ndf["new_links"].value_counts()

new_links
[]    488
Name: count, dtype: int64

In [51]:
train_df = pd.read_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_train_dsr1_ans_20240317_v2.xlsx')
train_df

,input,output
0,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,📺 **视频号直播间支持挂载广告插件**，具体配置方式和效果如下：\n\n---\n\n##...
1,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,### 关于“退市人民币”广告投放限制的说明及解决方案\n\n根据您提供的信息，当前蹊径落地...
2,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,根据您的账户记录显示，该笔资金转出是服务商之间的转账操作。以下是可能存在的转账原因及核查方式...
3,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,您好！关于在**软件开发/IT行业**推广**个人智能名片生成小程序**的投放建议如下：\n...
4,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,关于客户ID：36808601 蹊径ID：21571031的行业不符问题，结合电商行业审核要...
...,...,...
483,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,根据腾讯广告的定向策略，**广告投放主要基于用户的地理位置、年龄、性别、设备类型、兴趣行为等...
484,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,**关于品专广告使用竞品品牌词的解答：**\n\n根据腾讯广告政策，**品牌专区广告（品专）...
485,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,根据腾讯广告功能说明，关于微信再营销中的**排除已关注公众号功能**，以下是核心要点：\n\...
486,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的资料回答[用户...,您好，关于您提到的推广内容涉及K12培训问题，以下是具体解析和修改建议：\n\n---\n\...


In [53]:
df_test = pd.read_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_test_0220_n100.xlsx')
df_test

,上文对话历史,用户当前提问,Deepseek-R1结果,input
0,用户：图片素材规则\nQ：关于图片素材的规则，以下是一些基本要求：\n\n1. **清晰度*...,图片素材的尺寸和大小是多少,\n\n根据腾讯广告的最新规范，以下是图片素材的尺寸和大小要求：\n\n---\n\n###...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
1,用户：昨日账户日报\nQ：### 一、妙问智能分析\n\n- **整体表现**：账户 ID ...,如何提高在线预览限额？,\n\n根据腾讯广告的在线预览功能规则，单账户下可配置的不同微信号总数上限为200个，这是当...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
2,用户：投放app下载注册，可以自定义上传人群包排除吗\nQ：您好，您可以在广告-定向-自定义...,系统如何匹配相似人群？,\n\n腾讯广告系统通过以下方式实现相似人群匹配：\n\n1. **营销目标引导**\n通过...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
3,用户：投放app下载注册，可以自定义上传人群包排除吗\nQ：您好，您可以在广告-定向-自定义...,上传精准人群包定向，系统会学习分辨这些用户去拓展类似人群吗,\n\n📌 **关于上传精准人群包定向后系统是否会学习并拓展类似人群的解答：**\n\n--...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
4,用户：og\nQ：在营销领域，OG通常是指优化目标（Optimization Goal），例...,og在投放中起什么作用？,\n\n**OG（优化目标）在广告投放中的核心作用** \n\nOG（Optimizati...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
...,...,...,...,...
91,用户：我想要直客\nQ：您好，自助开户即为直客账户，如需申请，请参考如下入口及攻略：\n\n...,直客账户有哪些优惠？,\n\n📌 **直客账户的优惠政策主要涉及返点激励和返佣合作模式**，具体如下：\n\n--...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
92,用户：昨日账户日报\nQ：账户48458741在2025-02-18暂时没查询到有消耗信息，...,品牌广告可以看到流量预估吗,\n\n📌 **关于品牌广告流量预估的解答**：\n\n目前腾讯广告平台中，**设备品牌型号...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
93,用户：搜索广告计划中，出价方式选择cpc并设置出价后，那么在关键词中还需要在单独设置每个关键...,那如果计划的cpc出价是0.5元，关键词的出价是0.7元，在实际推广中，是参考0.5还是0.7,\n\n📌 **当广告计划CPC出价为0.5元，关键词出价为0.7元时，实际推广会优先按照关...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...
94,用户：搜索广告计划中，出价方式选择cpc并设置出价后，那么在关键词中还需要在单独设置每个关键...,cpc出价后关键词还要出价吗,\n\n📌 **关于CPC出价后关键词是否需要单独出价的解答：**\n\n---\n\n##...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...


In [59]:
import pandas as pd
import re
import requests
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urlparse

# 设置请求超时时间（秒）
REQUEST_TIMEOUT = 5

def check_link_accessibility(url):
    """检查单个链接的可访问性"""
    try:
        if "example.com" in url:
            return False, "示例域名"
        # 添加User-Agent头以避免被某些网站拒绝
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        # 只获取头部信息，不下载整个页面
        response = requests.head(
            url, 
            headers=headers, 
            timeout=REQUEST_TIMEOUT,
            allow_redirects=True
        )
        
        # 检查状态码是否为2xx或3xx
        if response.status_code < 400:
            return True, None
        else:
            return False, f"HTTP {response.status_code}"
    
    except requests.exceptions.RequestException as e:
        error_msg = str(e)
        if 'SSL' in error_msg:
            return False, "SSL错误"
        elif 'ConnectTimeout' in error_msg:
            return False, "连接超时"
        elif 'ConnectionError' in error_msg:
            return False, "连接错误"
        else:
            return False, error_msg
        
def check_links_accessibility(urls):
    """批量检查链接可访问性（使用多线程）"""
    if not urls:
        return [], []
    
    accessible = []
    inaccessible = []
    
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(check_link_accessibility, urls))
    
    for url, (is_accessible, error_msg) in zip(urls, results):
        if is_accessible:
            accessible.append(url)
        else:
            inaccessible.append((url, error_msg))
    
    return accessible, inaccessible

def extract_links(text):
    """从文本中提取所有超链接"""
    if pd.isna(text):
        return []
    
    # markdown_links = re.findall(r'(https?://[^\s)]+)', str(text))
    markdown_links = re.findall( r'[($$](https?://[^\s)$$]+)[)\]]', str(text))

    
    return markdown_links

# 从r1_answer列提取所有超链接
df_test['all_links'] = df_test['Deepseek-R1结果'].apply(extract_links)

# 检查每个链接是否出现在input列中
def check_links(row):
    if not row['extracted_links']:
        return True
    input_text = str(row['input']) if pd.notna(row['input']) else ''
    return all(link in input_text for link in row['extracted_links'])

def check_and_record_links(row):
    if not row['all_links']:
        return {'link_check': True, 'new_links': []}
    
    input_text = str(row['input']) if pd.notna(row['input']) else ''
    missing_links = [link for link in row['all_links'] if link not in input_text]
    
    # 检查缺失链接的可访问性
    accessible, inaccessible = check_links_accessibility(missing_links)
    
    return {
        'link_check': len(inaccessible) == 0,
        'new_links': missing_links,
        'accessible_links': accessible,
        'inaccessible_links': inaccessible
    }

result = df_test.apply(check_and_record_links, axis=1, result_type='expand')
df_test['link_check'] = result['link_check']
df_test['new_links'] = result['new_links']
df_test['accessible_links'] = result['accessible_links']
df_test['inaccessible_links'] = result['inaccessible_links']
# df['link_check'] = df.apply(check_links, axis=1)

# 删除临时列（如果需要）
# df = df.drop('extracted_links', axis=1)

df_test

,上文对话历史,用户当前提问,Deepseek-R1结果,input,all_links,link_check,new_links,accessible_links,inaccessible_links
0,用户：图片素材规则\nQ：关于图片素材的规则，以下是一些基本要求：\n\n1. **清晰度*...,图片素材的尺寸和大小是多少,\n\n根据腾讯广告的最新规范，以下是图片素材的尺寸和大小要求：\n\n---\n\n###...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/2...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2024/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/2...,"[(https://ads.tencent.com/, HTTPSConnectionPoo..."
1,用户：昨日账户日报\nQ：### 一、妙问智能分析\n\n- **整体表现**：账户 ID ...,如何提高在线预览限额？,\n\n根据腾讯广告的在线预览功能规则，单账户下可配置的不同微信号总数上限为200个，这是当...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,"[https://ad.qq.com, https://qcloudimg.tencent-...",False,"[https://ad.qq.com, https://qcloudimg.tencent-...",[https://ad.qq.com],[(https://qcloudimg.tencent-cloud.cn/raw/examp...
2,用户：投放app下载注册，可以自定义上传人群包排除吗\nQ：您好，您可以在广告-定向-自定义...,系统如何匹配相似人群？,\n\n腾讯广告系统通过以下方式实现相似人群匹配：\n\n1. **营销目标引导**\n通过...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://qcloudimg.tencent-cloud.cn/raw/7d3c9d...,False,[https://qcloudimg.tencent-cloud.cn/raw/7d3c9d...,[https://docs.qq.com/doc/DQWJ3V1JpR2dLZ0lF],[(https://qcloudimg.tencent-cloud.cn/raw/7d3c9...
3,用户：投放app下载注册，可以自定义上传人群包排除吗\nQ：您好，您可以在广告-定向-自定义...,上传精准人群包定向，系统会学习分辨这些用户去拓展类似人群吗,\n\n📌 **关于上传精准人群包定向后系统是否会学习并拓展类似人群的解答：**\n\n--...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/4...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2020/4...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/4...,[(https://bbx-pic.gtimg.com/bbx/pictures/2020/...
4,用户：og\nQ：在营销领域，OG通常是指优化目标（Optimization Goal），例...,og在投放中起什么作用？,\n\n**OG（优化目标）在广告投放中的核心作用** \n\nOG（Optimizati...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,"[https://example.com/doc-link, https://example...",False,"[https://example.com/doc-link, https://example...",[],"[(https://example.com/doc-link, 示例域名), (https:..."
...,...,...,...,...,...,...,...,...,...
91,用户：我想要直客\nQ：您好，自助开户即为直客账户，如需申请，请参考如下入口及攻略：\n\n...,直客账户有哪些优惠？,\n\n📌 **直客账户的优惠政策主要涉及返点激励和返佣合作模式**，具体如下：\n\n--...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://support.e.qq.com/detail?cid=4079&pid=...,True,[],[],[]
92,用户：昨日账户日报\nQ：账户48458741在2025-02-18暂时没查询到有消耗信息，...,品牌广告可以看到流量预估吗,\n\n📌 **关于品牌广告流量预估的解答**：\n\n目前腾讯广告平台中，**设备品牌型号...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,False,[https://support.e.qq.com/detail?cid=4230&pid=...,[https://support.e.qq.com/detail?cid=4230&pid=...,"[(https://example.com/trend-chart.png, 示例域名)]"
93,用户：搜索广告计划中，出价方式选择cpc并设置出价后，那么在关键词中还需要在单独设置每个关键...,那如果计划的cpc出价是0.5元，关键词的出价是0.7元，在实际推广中，是参考0.5还是0.7,\n\n📌 **当广告计划CPC出价为0.5元，关键词出价为0.7元时，实际推广会优先按照关...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://qcloudimg.tencent-cloud.cn/archive/3a...,False,[https://qcloudimg.tencent-cloud.cn/archive/3a...,[],[(https://qcloudimg.tencent-cloud.cn/archive/3...
94,用户：搜索广告计划中，出价方式选择cpc并设置出价后，那么在关键词中还需要在单独设置每个关键...,cpc出价后关键词还要出价吗,\n\n📌 **关于CPC出价后关键词是否需要单独出价的解答：**\n\n---\n\n##...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://ad.tencent.com/help-center],False,[https://ad.tencent.com/help-center],[],"[(https://ad.tencent.com/help-center, HTTPSCon..."


In [60]:
df_test['link_check'].value_counts()

link_check
True     52
False    44
Name: count, dtype: int64

In [61]:
df_test.to_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_test_0220_n100_checked.xlsx')

In [62]:
from tqdm import *

i = 0
for index, row in tqdm(df_test.iterrows()):
    if row['link_check'] == False:
        answer = row['Deepseek-R1结果']
        links = [ l[0] for l in row['inaccessible_links']]
        prompt = gen_prompt(answer, links)
        result = request_deepseek(prompt)
        if i < 3:
            print(result)
        i += 1
        df_test.at[index, 'answer_new'] = result
    else:
        df_test.at[index, 'answer_new'] = row['Deepseek-R1结果']
df_test

1it [00:16, 16.51s/it]

根据腾讯广告的最新规范，以下是图片素材的尺寸和大小要求：

---

### 一、通用图片规范
1. **基础尺寸要求**  
- 浮层卡片图片：支持 **800x800像素**  
- 商品图/LOGO/贴纸：支持PNG/JPG格式，建议分辨率不低于 **72dpi**  
- 图片图层：不支持智能对象图层或图层组  

---

### 二、不同版位尺寸差异
腾讯广告的素材尺寸因广告版位不同而异，例如：
- **朋友圈广告**：常见尺寸为 **1080x1080像素**（正方形）或 **1080x607像素**（横版）  
- **QQ广告**：需根据具体场景（如动态、聊天窗口）调整  
- **其他版位**：建议在投放管理平台创建广告时查看实时尺寸提示  

---

### 三、批量编辑与生成工具
1. **批量处理同尺寸素材**  
- 在腾讯广告妙思工具中，可批量选中多张同尺寸图片，一键应用设计效果。  

2. **AI生成拼接功能**  
- 编辑过程中支持通过“文生图”生成新素材，快速拼接创意。  

---

### 四、操作建议
1. **登录平台查看实时要求**  
在腾讯广告投放管理平台创建广告时，系统会展示当前版位的具体尺寸要求。  
2. **模板复用与保存**  
编辑操作可保存为模板，方便后续复用。  

若需更具体的版位尺寸（如朋友圈或QQ），请补充说明广告类型，我会进一步提供详细参数！


2it [00:25, 12.08s/it]

根据腾讯广告的在线预览功能规则，单账户下可配置的不同微信号总数上限为200个，这是当前系统设置的限制。若您遇到限额问题，可通过以下方式优化使用：

**操作建议：**
1. **清理历史绑定**  
👉 定期在账户管理后台检查并删除不再使用的微信号绑定，释放配额。

2. **分广告层级配置**  
👉 单广告下可绑定300个微信号，建议将不同微信号分散到多个广告下配置，充分利用广告层级的更高限额。

3. **动态调整时间**  
👉 若触发30分钟内300个账号的绑定上限，可等待30分钟后继续操作。

**功能操作指引：**  
（操作路径：广告列表 → 点击广告“操作”栏 → 选择“在线预览”）

目前系统暂未开放账户层级限额的人工调整服务，建议通过上述方式优化配置。如需进一步帮助，可提供具体使用场景，我们将协助排查问题。


3it [00:32,  9.87s/it]

腾讯广告系统通过以下方式实现相似人群匹配：

1. **营销目标引导**
通过广告后台设置明确的营销目标（如应用下载、商品销售等），系统会根据目标自动学习商品/服务的转化路径特征。

2. **机器学习模型**
系统基于历史转化数据，通过深度神经网络算法分析高转化人群的特征，包括但不限于：
- 用户行为轨迹（页面停留、点击偏好）
- 消费特征（价格敏感度、品类偏好）
- 设备属性（机型、网络环境）

3. **实时动态优化**
每6分钟更新一次人群特征库，根据实时转化数据调整投放策略，动态扩展相似人群范围。

建议在投放时保持3天以上的稳定投放期，为系统提供充足的学习数据。


96it [08:31,  5.33s/it]


,上文对话历史,用户当前提问,Deepseek-R1结果,input,all_links,link_check,new_links,accessible_links,inaccessible_links,answer_new
0,用户：图片素材规则\nQ：关于图片素材的规则，以下是一些基本要求：\n\n1. **清晰度*...,图片素材的尺寸和大小是多少,\n\n根据腾讯广告的最新规范，以下是图片素材的尺寸和大小要求：\n\n---\n\n###...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/2...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2024/2...,[https://bbx-pic.gtimg.com/bbx/pictures/2024/2...,"[(https://ads.tencent.com/, HTTPSConnectionPoo...",根据腾讯广告的最新规范，以下是图片素材的尺寸和大小要求：\n\n---\n\n### 一、通...
1,用户：昨日账户日报\nQ：### 一、妙问智能分析\n\n- **整体表现**：账户 ID ...,如何提高在线预览限额？,\n\n根据腾讯广告的在线预览功能规则，单账户下可配置的不同微信号总数上限为200个，这是当...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,"[https://ad.qq.com, https://qcloudimg.tencent-...",False,"[https://ad.qq.com, https://qcloudimg.tencent-...",[https://ad.qq.com],[(https://qcloudimg.tencent-cloud.cn/raw/examp...,根据腾讯广告的在线预览功能规则，单账户下可配置的不同微信号总数上限为200个，这是当前系统设...
2,用户：投放app下载注册，可以自定义上传人群包排除吗\nQ：您好，您可以在广告-定向-自定义...,系统如何匹配相似人群？,\n\n腾讯广告系统通过以下方式实现相似人群匹配：\n\n1. **营销目标引导**\n通过...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://qcloudimg.tencent-cloud.cn/raw/7d3c9d...,False,[https://qcloudimg.tencent-cloud.cn/raw/7d3c9d...,[https://docs.qq.com/doc/DQWJ3V1JpR2dLZ0lF],[(https://qcloudimg.tencent-cloud.cn/raw/7d3c9...,腾讯广告系统通过以下方式实现相似人群匹配：\n\n1. **营销目标引导**\n通过广告后台...
3,用户：投放app下载注册，可以自定义上传人群包排除吗\nQ：您好，您可以在广告-定向-自定义...,上传精准人群包定向，系统会学习分辨这些用户去拓展类似人群吗,\n\n📌 **关于上传精准人群包定向后系统是否会学习并拓展类似人群的解答：**\n\n--...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/4...,False,[https://bbx-pic.gtimg.com/bbx/pictures/2020/4...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/4...,[(https://bbx-pic.gtimg.com/bbx/pictures/2020/...,📌 **关于上传精准人群包定向后系统是否会学习并拓展类似人群的解答：**\n\n---\n\...
4,用户：og\nQ：在营销领域，OG通常是指优化目标（Optimization Goal），例...,og在投放中起什么作用？,\n\n**OG（优化目标）在广告投放中的核心作用** \n\nOG（Optimizati...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,"[https://example.com/doc-link, https://example...",False,"[https://example.com/doc-link, https://example...",[],"[(https://example.com/doc-link, 示例域名), (https:...",**OG（优化目标）在广告投放中的核心作用** \n\nOG（Optimization G...
...,...,...,...,...,...,...,...,...,...,...
91,用户：我想要直客\nQ：您好，自助开户即为直客账户，如需申请，请参考如下入口及攻略：\n\n...,直客账户有哪些优惠？,\n\n📌 **直客账户的优惠政策主要涉及返点激励和返佣合作模式**，具体如下：\n\n--...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://support.e.qq.com/detail?cid=4079&pid=...,True,[],[],[],\n\n📌 **直客账户的优惠政策主要涉及返点激励和返佣合作模式**，具体如下：\n\n--...
92,用户：昨日账户日报\nQ：账户48458741在2025-02-18暂时没查询到有消耗信息，...,品牌广告可以看到流量预估吗,\n\n📌 **关于品牌广告流量预估的解答**：\n\n目前腾讯广告平台中，**设备品牌型号...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://bbx-pic.gtimg.com/bbx/pictures/2020/8...,False,[https://support.e.qq.com/detail?cid=4230&pid=...,[https://support.e.qq.com/detail?cid=4230&pid=...,"[(https://example.com/trend-chart.png, 示例域名)]",📌 **关于品牌广告流量预估的解答**：\n\n目前腾讯广告平台中，**设备品牌型号定向暂不...
93,用户：搜索广告计划中，出价方式选择cpc并设置出价后，那么在关键词中还需要在单独设置每个关键...,那如果计划的cpc出价是0.5元，关键词的出价是0.7元，在实际推广中，是参考0.5还是0.7,\n\n📌 **当广告计划CPC出价为0.5元，关键词出价为0.7元时，实际推广会优先按照关...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://qcloudimg.tencent-cloud.cn/archive/3a...,False,[https://qcloudimg.tencent-cloud.cn/archive/3a...,[],[(https://qcloudimg.tencent-cloud.cn/archive/3...,📌 **当广告计划CPC出价为0.5元，关键词出价为0.7元时，实际推广会优先按照关键词出价...
94,用户：搜索广告计划中，出价方式选择cpc并设置出价后，那么在关键词中还需要在单独设置每个关键...,cpc出价后关键词还要出价吗,\n\n📌 **关于CPC出价后关键词是否需要单独出价的解答：**\n\n---\n\n##...,你是腾讯广告客服人员，回答腾讯广告相关的[用户问题]，参考[广告资料]里提供的最新知识回答[...,[https://ad.tencent.com/help-center],False,[https://ad.tencent.com/help-center],[],"[(https://ad.tencent.com/help-center, HTTPSCon...",📌 **关于CPC出价后关键词是否需要单独出价的解答：**\n\n---\n\n### **...


In [63]:
df_test.to_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_test_0220_n100_remove_hallucination_link.xlsx', index=False)

In [64]:
df_test["output"] = df_test["answer_new"]
df_test[["input", "output"]].to_excel('/Users/wenweili/Desktop/AI客服/RAG训练/data/rag_test_0220_n100_v1.xlsx', index=False)

In [74]:
df_test['input_length'] = df_test['input'].apply(len)

# 统计长度分布
length_distribution = df_test['input_length'].value_counts().sort_index()

print("Input 列的长度分布：")
print(length_distribution)

Input 列的长度分布：
input_length
353     2
354     2
357     1
358     2
359     1
       ..
5598    1
6072    1
6312    1
7965    1
8504    1
Name: count, Length: 90, dtype: int64


In [73]:
from similarities import BertSimilarity
m = BertSimilarity(model_name_or_path="shibing624/text2vec-base-chinese")
r = m.similarity('如何更换花呗绑定银行卡', '花呗更改绑定银行卡')
print(f"similarity score: {float(r)}")  # similarity score: 0.855146050453186

tokenizer_config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

2025-04-16 16:14:21.959 | DEBUG    | text2vec.sentence_model:__init__:78 - Use pytorch device: mps


similarity score: 0.8551464676856995


In [71]:
!pip install similarities

  Using cached similarities-1.2.3-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 314.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached jieba-0.42.1-py3-none-any.whl
  Obtaining dependency information for loguru from https://files.pythonhosted.org/packages/0c/29/0348de65b8cc732daa3e33e67806420b2ae89bdce2b04af740289c5c6c8c/loguru-0.7.3-py3-none-any.whl.metadata
  Using cached fire-0.7.0.tar.gz (87 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for autofaiss from https://files.pythonhosted.org/packages/3b/39/9a9339c0dcbea945f6fc5df3dab9a262bcecd0af7a541bc91b222cec4822/autofaiss-2.17.0-py3-none-any.whl.metadata
  Using cached autofaiss-2.17.0-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/e4/0d/d60f023abd74e1ccd448c97ec9c0d78ddc43a95497c14939a05c5de6f887/gensim-4.3.3-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Using cach